Examples are provided on how to correct astronomical observations for
atmospheric effects. For demonstration purposes, the following examples
exclusively consider correcting observations of a black body.

## Spectrographic Observations

In order to correct specrographic observations for atmospheric effects,
observed spectra are divided by the atmospheric transmission function. As
an example, consider the spectral energy distribution (SED) of a black body
under the effects of atmospheric absorption. For a temperature of 8,000 K, and
a PWV level of 15 mm, this can be found as

In [1]:
from pwv_kpno import blackbody_with_atm as bb_atm
import numpy as np

temp = 8000
wavelengths = np.arange(7000, 10000, 100)
pwv = 15

sed = bb_atm.sed(temp, wavelengths, pwv)
print(sed)


[67943236.69609079 70958962.41142142 62627486.36859217 62880671.63407001
 62513485.51967486 63058747.67338449 61110444.36407656 59221775.73675153
 57395177.10964452 55513103.21142037 51811690.73561192 51911555.0857156
 44194129.49981948 48607782.58147565 41482678.59910741 46144046.17659909
 44824567.81888515 43492384.1617855  42199645.17754546 40860832.93858474
 38511276.79340703 35308534.66156565 35854592.31322291 13213391.88383076
 31739851.68482697  3175256.11453751 26648422.40842594 28300788.53752817
 30913697.98483177 29947238.79680985]


Since we know the PWV level used to generate this spectra, we can determine the
corresponding transmission function using the ``trans_for_pwv`` function

In [2]:
from pwv_kpno import pwv_atm

modeled_trans = pwv_atm.trans_for_pwv(15)
print(modeled_trans)

    wavelength        transmission   
     Angstrom              %         
------------------ ------------------
            7000.0 0.9198165020431311
 7001.000333444482 0.8590254136327501
 7002.000666888963 0.9967736288238565
 7003.001000333445 0.9934494271823038
7004.0013337779255 0.9607896418908324
 7005.001667222407 0.9947675112942339
 7006.002000666889 0.9887061420963239
  7007.00233411137 0.8805365960557212
 7008.002667555852 0.9708917349052267
 7009.003001000334 0.9925593320994702
               ...                ...
 9990.996998999666 0.9797313896855093
 9991.997332444149 0.9964175769435284
  9992.99766588863 0.9965329454238108
  9993.99799933311 0.9991854372141895
 9994.998332777592 0.9977433129559928
 9995.998666222074 0.9811599116724874
 9996.998999666555 0.9943027771012202
 9997.999333111036 0.9987372729496821
 9998.999666555519  0.999357354753788
           10000.0 0.9990180076538167
Length = 3000 rows


In order to divide these two results, the SED at and the transmission function
must both be known for the same wavelength values. Since the SED is a well
behaved function, we interpolate the SED to match the wavelength sampling of
the transmission function. In general the transmission function not a smooth
function,which can yield innacuracies when interpolating.

Using the ``numpy`` package, we interpolate as follows:


In [7]:
sampled_sed = np.interp(modeled_trans["wavelength"],
                        wavelengths,
                        sed)


The corrected spectrum can then be found by dividing the observed flux
by the transmission on a wavelength by wavelength basis.


In [8]:
corrected_spec = np.divide(sampled_sed, modeled_trans["transmission"])

In [10]:
i_band = (7000, 8500)

weighted_transmission = np.multiply(sampled_sed, modeled_trans["transmission"])
numerator = np.trapz(weighted_transmission, i_band)
denominator = np.trapz(sampled_sed, i_band)
photo_corr = np.divide(numerator, denominator)